## BMI Calculator

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from IPython.display import Image

In [ ]:
# Define state
class BMIState(TypedDict):

    weight_kg: float
    height_m: float
    bmi: float
    category: str

In [ ]:
def calculate_bmi(state: BMIState) -> BMIState:
    weight = state['weight_kg']
    height = state["height_m"]

    bmi = weight/(height**2)

    state["bmi"] = round(bmi, 2)

    return state

In [ ]:
def label_bmi(state: BMIState) -> BMIState:
    bmi = state["bmi"]

    if bmi < 18.5:
        state["category"] = "Underweight"
    elif 18.5 <= bmi < 25:
        state["category"] = "Normal"
    elif 25 <= bmi < 30:
        state["category"] = "Overweight"
    else:
        state["category"] = "Obese"

    return state


In [ ]:
# Define Graph
graph = StateGraph(BMIState)

# Add nodes to graph
graph.add_node('calculate_bmi', calculate_bmi)
graph.add_node('label_bmi', label_bmi)

# Add edges to graph
graph.add_edge(START, 'calculate_bmi')
graph.add_edge('calculate_bmi', 'label_bmi')
graph.add_edge('label_bmi', END)

# Compile the graph 
workflow = graph.compile()

In [ ]:
# Execute the graph
initial_state = {"weight_kg": 64, "height_m": 1.68}
final_state = workflow.invoke(initial_state)
print(final_state)

In [ ]:
Image(workflow.get_graph().draw_mermaid_png())

## LLM Workflow

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_groq import ChatGroq
from IPython.display import Image
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
model = llm = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2
)

In [ ]:
# Define State

class LLMState(TypedDict):

    question: str
    answer: str

In [ ]:
def llm_qa(state: LLMState) -> LLMState:
    
    # Extract the question from state
    question = state["question"]

    # Form a prompt
    prompt = f"Answer the following question {question}"

    # Ask the questions
    answer = model.invoke(prompt).content

    # Update the answer in the state

    state["answer"] = answer

    return state

In [ ]:
# Define the Graph
graph = StateGraph(LLMState)

# Add nodes to Graph
graph.add_node("llm_qa", llm_qa)

# Add edges to the graph
graph.add_edge(START, "llm_qa")
graph.add_edge("llm_qa", END)

# Compile
workflow = graph.compile()

In [ ]:
Image(workflow.get_graph().draw_mermaid_png())

In [ ]:
initial_state = {"question": "How far is moon from the Earth?"}

final_state = workflow.invoke(initial_state)
print(final_state["answer"])

## Pompt Chaining

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from IPython.display import Image
from dotenv import load_dotenv
from langchain_groq import ChatGroq

In [ ]:
load_dotenv()

In [ ]:
model = llm = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2
)

In [ ]:
# Define State

class BlogState(TypedDict):

    title: str
    outline: str
    content: str

In [ ]:
def create_outline(state: BlogState) -> BlogState:

    # fetch title
    title = state["title"]

    # call llm gen outline
    prompt = f"Generate a detailed outline for a blog on the topic - {title}"
    outline = model.invoke(prompt).content

    # Update state

    state["outline"] = outline

    return state

In [ ]:
def create_blog(state: BlogState) -> BlogState:

    # fetch title
    title = state["title"]
    outline = state["outline"]

    prompt = f"Write a etailed blog on the title - {title} using the following outline \n {outline}"
    content = model.invoke(prompt).content

    # Update state

    state["content"] = content

    return state

In [ ]:
# Define graph

graph = StateGraph(BlogState)

# Define Nodes
graph.add_node("create_outline", create_outline)
graph.add_node("create_blog", create_blog)

# Define Edges
graph.add_edge(START, "create_outline")
graph.add_edge("create_outline", "create_blog")
graph.add_edge("create_blog", END)


# Compile
workflow = graph.compile()


In [ ]:
Image(workflow.get_graph().draw_mermaid_png())

In [ ]:
initial_state = {"title": "Rise of AI in Bangladesh"}
final_state = workflow.invoke(initial_state)
print(final_state)

In [ ]:
print(final_state["outline"])

In [ ]:
print(final_state["content"])

## Parallel Workflows in LangGraph

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

In [ ]:
# Define State

class AnalysisPlayerPerformanceState(TypedDict):

    runs: int
    balls: int
    fours: int
    sixes: int
    strike_rate: float
    balls_per_boundary: float
    boundary_percent: float
    summary: str


In [ ]:
def calculate_sr(state: AnalysisPlayerPerformanceState):

    sr = (state["runs"]/state["balls"])*100

    return {"strike_rate": sr}

In [ ]:
def calculate_bpb(state: AnalysisPlayerPerformanceState):

    bpb = (state["balls"]/state["fours"] + state["sixes"])

    return {"balls_per_boundary": bpb}

In [ ]:
def calculate_boundary_percent(state: AnalysisPlayerPerformanceState):

    boundary_percent = (((state["fours"]*4) + (state["sixes"]*6))/state["runs"])*100

    return {"boundary_percent": boundary_percent}


In [ ]:
def summary(state: AnalysisPlayerPerformanceState):

    summary = f"""
    Strike Rate - {state["strike_rate"]} \n
    Balls per boundary - {state["balls_per_boundary"]} \n
    Boundary percent - {state["boundary_percent"]}
    """

    return {"summary": summary}

In [ ]:
# Define Graph
graph = StateGraph(AnalysisPlayerPerformanceState)

# Add Node
graph.add_node("calculate_sr", calculate_sr)
graph.add_node("calculate_bpb", calculate_bpb)
graph.add_node("calculate_boundary_percent", calculate_boundary_percent)
graph.add_node("summary", summary)

# Add Edge
graph.add_edge(START, 'calculate_sr')
graph.add_edge(START, 'calculate_bpb')
graph.add_edge(START, 'calculate_boundary_percent')

graph.add_edge("calculate_sr", 'summary')
graph.add_edge("calculate_bpb", 'summary')
graph.add_edge("calculate_boundary_percent", 'summary')

graph.add_edge("calculate_boundary_percent", END)

# Compile
workflow = graph.compile()

In [ ]:
Image(workflow.get_graph().draw_mermaid_png())

In [ ]:
initial_state = {
    "runs": 100,
    "balls": 89,
    "fours": 6,
    "sixes": 3
}

final_state = workflow.invoke(initial_state)

In [ ]:
print(final_state)

## Essay Evaluations

In [ ]:
from langgraph.graph import StateGraph,START, END
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from typing import TypedDict, Annotated
from pydantic import BaseModel, Field
import operator

In [ ]:
load_dotenv()

In [ ]:
model = llm = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2
)

In [ ]:
class EvaluationSchema(BaseModel):

    feedback: str = Field(description="Detailed feedback for the essay")
    score: int = Field(description="Score out of 10", ge = 0, le = 10)

In [ ]:
structured_model = model.with_structured_output(EvaluationSchema)

###########################################################

In [ ]:
essay = """Bangladesh in the Age of AI
As the world enters a transformative era defined by artificial intelligence (AI), Bangladesh stands at a critical juncture — one where it can either emerge as a global leader in AI innovation or risk falling behind in the technology race. The age of AI brings with it immense promise as well as unprecedented challenges, and how Bangladesh navigates this landscape will shape its socio-economic and geopolitical future.

Bangladesh's strengths in the AI domain are rooted in its vast pool of skilled engineers, a thriving IT industry, and a growing startup ecosystem. With over 5 million STEM graduates annually and a burgeoning base of AI researchers, Bangladesh possesses the intellectual capital required to build cutting-edge AI systems. Institutions like IITs, IIITs, and IISc have begun fostering AI research, while private players such as TCS, Infosys, and Wipro are integrating AI into their global services. In 2020, the government launched the National AI Strategy (AI for All) with a focus on inclusive growth, aiming to leverage AI in healthcare, agriculture, education, and smart mobility.

One of the most promising applications of AI in Bangladesh lies in agriculture, where predictive analytics can guide farmers on optimal sowing times, weather forecasts, and pest control. In healthcare, AI-powered diagnostics can help address Bangladesh’s doctor-patient ratio crisis, particularly in rural areas. Educational platforms are increasingly using AI to personalize learning paths, while smart governance tools are helping improve public service delivery and fraud detection.

However, the path to AI-led growth is riddled with challenges. Chief among them is the digital divide. While metropolitan cities may embrace AI-driven solutions, rural Bangladesh continues to struggle with basic internet access and digital literacy. The risk of job displacement due to automation also looms large, especially for low-skilled workers. Without effective skilling and re-skilling programs, AI could exacerbate existing socio-economic inequalities.

Another pressing concern is data privacy and ethics. As AI systems rely heavily on vast datasets, ensuring that personal data is used transparently and responsibly becomes vital. Bangladesh is still shaping its data protection laws, and in the absence of a strong regulatory framework, AI systems may risk misuse or bias.

To harness AI responsibly, Bangladesh must adopt a multi-stakeholder approach involving the government, academia, industry, and civil society. Policies should promote open datasets, encourage responsible innovation, and ensure ethical AI practices. There is also a need for international collaboration, particularly with countries leading in AI research, to gain strategic advantage and ensure interoperability in global systems.

Bangladesh’s demographic dividend, when paired with responsible AI adoption, can unlock massive economic growth, improve governance, and uplift marginalized communities. But this vision will only materialize if AI is seen not merely as a tool for automation, but as an enabler of human-centered development.

In conclusion, Bangladesh in the age of AI is a story in the making — one of opportunity, responsibility, and transformation. The decisions we make today will not just determine Bangladesh’s AI trajectory, but also its future as an inclusive, equitable, and innovation-driven society."""

In [ ]:
prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {essay}'
structured_model.invoke(prompt)

In [ ]:
structured_model.invoke(prompt).feedback

In [ ]:
structured_model.invoke(prompt).score

#############################################################

In [ ]:
class UPSCState(TypedDict):
    
    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_scores: Annotated[list[int], operator.add]
    avg_score: float

In [ ]:
def evaluate_language(state: UPSCState):

    prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)

    return {'language_feedback': output.feedback, 'individual_scores': [output.score]}

In [ ]:
def evaluate_analysis(state: UPSCState):

    prompt = f'Evaluate the depth of analysis of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)

    return {'analysis_feedback': output.feedback, 'individual_scores': [output.score]}

In [ ]:
def evaluate_thought(state: UPSCState):

    prompt = f'Evaluate the clarity of thought of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)

    return {'clarity_feedback': output.feedback, 'individual_scores': [output.score]}

In [ ]:
def final_evaluation(state: UPSCState):

    # summary feedback
    prompt = f'Based on the following feedbacks create a summarized feedback \n language feedback - {state["language_feedback"]} \n depth of analysis feedback - {state["analysis_feedback"]} \n clarity of thought feedback - {state["clarity_feedback"]}'
    overall_feedback = model.invoke(prompt).content

    # avg calculate
    avg_score = sum(state['individual_scores'])/len(state['individual_scores'])

    return {'overall_feedback': overall_feedback, 'avg_score': avg_score}
    

In [ ]:
# Define Graph
graph = StateGraph(UPSCState)

# Add Node
graph.add_node("evaluate_language", evaluate_language)
graph.add_node("evaluate_analysis", evaluate_analysis)
graph.add_node("evaluate_thought", evaluate_thought)
graph.add_node("final_evaluation", final_evaluation)


# Add Edge
graph.add_edge(START, 'evaluate_language')
graph.add_edge(START, 'evaluate_analysis')
graph.add_edge(START, 'evaluate_thought')

graph.add_edge('evaluate_language', 'final_evaluation')
graph.add_edge('evaluate_analysis', 'final_evaluation')
graph.add_edge('evaluate_thought', 'final_evaluation')

graph.add_edge('final_evaluation', END)

workflow = graph.compile()

In [ ]:
workflow

In [ ]:
essay2 = """Bangladesh and AI Time

Now world change very fast because new tech call Artificial Intel… something (AI). Bangladesh also want become big in this AI thing. If work hard, Bangladesh can go top. But if no careful, Bangladesh go back.

Bangladesh have many good. We have smart student, many engine-ear, and good IT peoples. Big company like TCS, Infosys, Wipro already use AI. Government also do program “AI for All”. It want AI in farm, doctor place, school and transport.

In farm, AI help farmer know when to put seed, when rain come, how stop bug. In health, AI help doctor see sick early. In school, AI help student learn good. Government office use AI to find bad people and work fast.

But problem come also. First is many villager no have phone or internet. So AI not help them. Second, many people lose job because AI and machine do work. Poor people get more bad.

One more big problem is privacy. AI need big big data. Who take care? Bangladesh still make data rule. If no strong rule, AI do bad.

Bangladesh must all people together – govern, school, company and normal people. We teach AI and make sure AI not bad. Also talk to other country and learn from them.

If Bangladesh use AI good way, we become strong, help poor and make better life. But if only rich use AI, and poor no get, then big bad thing happen.

So, in short, AI time in Bangladesh have many hope and many danger. We must go right road. AI must help all people, not only some. Then Bangladesh grow big and world say "good job Bangladesh"."""


In [ ]:
intial_state = {
    'essay': essay2
}

workflow.invoke(intial_state)